The trip mode choice models explanatory variables include household and person variables, level-of-service between the trip origin and destination according to the time period for the tour leg, urban form variables, and alternative-specific constants segmented by tour mode.

In [1]:
import os
import toml
import pandas as pd
import numpy as np
import plotly.express as px
import validation_data_input
import psrc_theme


import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected" # to show plotly figures in quarto HTML file
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
# read data
land_use = pd.read_csv(config['p_survey_landuse'])

per_data = validation_data_input.get_persons_data(['ptype'])
hh_data = validation_data_input.get_households_data(['auto_ownership','home_zone_id'])
tour_data = validation_data_input.get_tours_data(['tour_mode', 'tour_type'])
trip_data = validation_data_input.get_trips_data()


# add auto_ownership with 4+
hh_data['auto_ownership_simple'] = hh_data['auto_ownership'].apply(lambda x: "4+" if x>=4.0 else str(x))
# household density groups
var_group = land_use['log_hh_1'].quantile([.00, .125, .25, .50, .75,1.00])
land_use['household_density_bin'] = pd.cut(land_use['log_hh_1'], bins=var_group.tolist(),
                                           labels=['very low', 'low', 'medium', 'medium-high', 'high'])

# add person type labels
ptype_cat = {1: "1: Full-Time Worker",
             2: "2: Part-Time Worker",
             3: "3: University Student",
             4: "4: Non-Working Adult Age <65",
             5: "5: Non-Working Adult Age 65+",
             6: "6: High School Student Age 16+",
             7: "7: Child Age 5-15",
             8: "8: Child Age 0-4"}
per_data['ptype_label'] = per_data['ptype'].map(ptype_cat)

hh_data = hh_data.merge(land_use[['log_hh_1','household_density_bin','zone_id']],how="left",left_on='home_zone_id',right_on='zone_id')
per_data = per_data.merge(hh_data[['household_id','auto_ownership','auto_ownership_simple','log_hh_1','household_density_bin','source']],
                          how='left', on=['household_id','source']) # get auto ownership from hh data

tour_data = tour_data.merge(per_data, how='left', on=['person_id','household_id','source'])

trip_data = trip_data.merge(tour_data, how='left',
                            on=['person_id','household_id','tour_id','trip_weight_2017_2019','source'])

- match values trip purpose and trip mode

In [3]:
#| echo: true
trip_data['purpose_match'] = trip_data['purpose'].apply(lambda x: "home" if x == "Home" else x)
trip_data['trip_mode_match'] = trip_data['trip_mode'].apply(lambda x: "TNC" if x == "TNC_SINGLE" else x)

In [4]:
# aggregate transit modes
transit_modes = ['WALK_LOC','WALK_COM','WALK_FRY','WALK_LR','DRIVE_TRN']
trip_mode_ordered = ["DRIVEALONEFREE", "SHARED2FREE", "SHARED3FREE", "BIKE","WALK","ALL_TRANSIT","SCH_BUS","TNC","Other"]
trip_data['trip_mode_transit_agg'] = trip_data['trip_mode_match'].apply(lambda x: "ALL_TRANSIT" if x in transit_modes else x)

df_plot = trip_data.groupby(['source','trip_mode_transit_agg'])['trip_weight_2017_2019'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight_2017_2019']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="trip_mode_transit_agg", y="percentage", color="source",barmode="group",
             category_orders={"trip_mode_transit_agg": trip_mode_ordered},
             title="Trip mode choice: all modes")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".1%"))
fig.show()

In [5]:
# show only transit modes
df_plot = trip_data.groupby(['source','trip_mode_match'])['trip_weight_2017_2019'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight_2017_2019']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot.loc[df_plot['trip_mode_match'].isin(transit_modes)], x="trip_mode_match", y="percentage", color="source",barmode="group",
             title="Trip mode choice: transit modes")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".1%"))
fig.show()

## Trip mode choice by segment

In [6]:
def plot_mode_choice(df: pd.DataFrame, grp_var: str, n_nol: int, height: int):
    df_plot = df.groupby(['source',grp_var,'trip_mode_transit_agg'])['trip_weight_2017_2019'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',grp_var], group_keys=False)['trip_weight_2017_2019']. \
        apply(lambda x: x / float(x.sum()))

    fig = px.bar(df_plot,
                 x="percentage", y="trip_mode_transit_agg", color="source",barmode="group",
                 facet_col=grp_var, facet_col_wrap=n_nol, orientation='h',
                 category_orders={"trip_mode_transit_agg": trip_mode_ordered},
                 title="Trip mode choice by " + grp_var)
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.update_layout(height=height, width=700, xaxis1=dict(tickformat=".0%"), xaxis2=dict(tickformat=".0%")
                      )
    fig.show()
def plot_mode_choice_transit(df: pd.DataFrame,  grp_var: str, n_nol: int, height: int):
    df_plot = df.groupby(['source',grp_var,'trip_mode_match'])['trip_weight_2017_2019'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',grp_var], group_keys=False)['trip_weight_2017_2019']. \
        apply(lambda x: x / float(x.sum()))

    fig = px.bar(df_plot.loc[df_plot['trip_mode_match'].isin(transit_modes)],
                 x="percentage", y="trip_mode_match", color="source",barmode="group",
                 facet_col=grp_var, facet_col_wrap=n_nol, orientation='h',
                 category_orders={grp_var: pd.Series(df_plot[grp_var].unique()).sort_values().to_list()},
                 title="Trip mode choice by " + grp_var + ": disaggregated transit modes")
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.update_layout(height=height, width=700, xaxis1=dict(tickformat=".1%"), xaxis2=dict(tickformat=".1%")
                      )
    fig.show()

In [7]:
plot_mode_choice(trip_data,'purpose_match',3,1200)
plot_mode_choice_transit(trip_data,'purpose_match',3,800)

In [8]:
plot_mode_choice(trip_data,'ptype_label',2,1200)
plot_mode_choice_transit(trip_data,'ptype_label',3,800)

In [9]:
plot_mode_choice(trip_data,'household_density_bin',3,600)
plot_mode_choice_transit(trip_data,'household_density_bin',3,500)

In [10]:
plot_mode_choice(trip_data,'auto_ownership_simple',3,600)
plot_mode_choice_transit(trip_data,'auto_ownership_simple',3,500)